# Use BoTorch for feasibility weighted acquisition function

import libearies

In [1]:
import pandas as pd
import numpy as np
import torch
import botorch
import gpytorch
import matplotlib.pyplot as plt

#modules for regression
from botorch.models.gp_regression import SingleTaskGP
from botorch.models.model_list_gp_regression import ModelListGP
from botorch.models.transforms.outcome import Standardize
from gpytorch.mlls.sum_marginal_log_likelihood import SumMarginalLogLikelihood
from botorch.utils.transforms import unnormalize, normalize

#modules for BO
from botorch.optim.optimize import optimize_acqf, optimize_acqf_list
from botorch.acquisition.objective import GenericMCObjective
from botorch.utils.multi_objective.scalarization import get_chebyshev_scalarization
from botorch.utils.multi_objective.box_decompositions.non_dominated import (
    FastNondominatedPartitioning,
)
from botorch.acquisition.multi_objective.monte_carlo import (
    qExpectedHypervolumeImprovement,
    qNoisyExpectedHypervolumeImprovement,
)
from botorch.utils.sampling import sample_simplex
from botorch import fit_gpytorch_mll
from botorch.exceptions import BadInitialCandidatesWarning
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.utils.multi_objective.box_decompositions.dominated import (
    DominatedPartitioning,
)
from botorch.utils.multi_objective.pareto import is_non_dominated

from gpytorch.likelihoods import DirichletClassificationLikelihood
from botorch.acquisition.multi_objective.objective import IdentityMCMultiOutputObjective


c:\Users\goura\anaconda3\envs\botorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Read and normalize data

In [2]:
filename = r'../data/olhs_run1.xlsx'
x_pd = pd.read_excel(filename, sheet_name='Initial Design (OLHS)', header=[0,1], index_col=[0])
y_pd = pd.read_excel(filename, sheet_name='bo_data', header=[0,1], index_col=[0])

dtype=torch.double

objective_properties = ['Polymer Solubility', 'Gelation Enthalpy', 'Shear Modulus']

x = torch.tensor(x_pd.values, dtype=dtype)
y = torch.tensor(y_pd[objective_properties].values, dtype=dtype)
mfg = torch.tensor(y_pd['Manufacturability'].values, dtype=torch.long)

x_bounds = np.array([[2000, 10000], [0, 100], [0, 40], [5000, 15000], [80, 100], [0,100], [60, 100], [70, 100]])
x_bounds = torch.tensor(x_bounds.T, dtype=dtype)

x = normalize(x, bounds=x_bounds)

In [5]:
y_all = torch.cat((y, mfg), dim=1)

Helper functions

Define and initialize regression and classification model

In [3]:
# Regression model
models = []
for i in range(len(objective_properties)):
    models.append(SingleTaskGP(x, y[:, i].unsqueeze(-1), 
                               outcome_transform=Standardize(m=1)))

# transformation of mfg labels
tmp_likl = DirichletClassificationLikelihood(targets=mfg.squeeze(), alpha_epsilon=1e-4, 
                                             learn_additional_noise=False)
cls_model = SingleTaskGP(train_X=x, train_Y=-tmp_likl.transformed_targets.T.double(), 
                         outcome_transform=Standardize(m=2))

models.append(cls_model)

model = ModelListGP(*models)
mll = SumMarginalLogLikelihood(model.likelihood, model)
mll = mll.to(x)

Fit the models to data

In [4]:
# regression model
fit_gpytorch_mll(mll)

SumMarginalLogLikelihood(
  (likelihood): LikelihoodList(
    (likelihoods): ModuleList(
      (0-3): 4 x GaussianLikelihood(
        (noise_covar): HomoskedasticNoise(
          (noise_prior): GammaPrior()
          (raw_noise_constraint): GreaterThan(1.000E-04)
        )
      )
    )
  )
  (model): ModelListGP(
    (models): ModuleList(
      (0-3): 4 x SingleTaskGP(
        (likelihood): GaussianLikelihood(
          (noise_covar): HomoskedasticNoise(
            (noise_prior): GammaPrior()
            (raw_noise_constraint): GreaterThan(1.000E-04)
          )
        )
        (mean_module): ConstantMean()
        (covar_module): ScaleKernel(
          (base_kernel): MaternKernel(
            (lengthscale_prior): GammaPrior()
            (raw_lengthscale_constraint): Positive()
          )
          (outputscale_prior): GammaPrior()
          (raw_outputscale_constraint): Positive()
        )
        (outcome_transform): Standardize()
      )
    )
    (likelihood): LikelihoodList

Prediction for sanity check

In [5]:
model.eval()

with torch.no_grad():
    posterior = model.posterior(x)
    pred_mean = posterior.mean
pred_mean

tensor([[ 53.0025,   4.3507,   5.1600,  13.3763,   0.7857],
        [ 23.5312,  54.3269,   1.0192,   1.2660,  12.8960],
        [ 38.3127,   6.4286,   1.5165,   1.2760,  12.8860],
        [ 31.9368,   5.7843,   1.4786,   1.2722,  12.8898],
        [120.8128,   3.3749,   2.3164,  13.4369,   0.7251],
        [ 81.0235,  83.1740,   1.5724,  13.4353,   0.7267],
        [ 84.5024,   3.5945,   2.7505,  13.3966,   0.7654],
        [ 65.4618,  28.0365,   3.4917,  13.3777,   0.7843],
        [ 49.5438, 107.8939,   5.2615,  13.3980,   0.7640],
        [167.9699,   4.9255,   9.9088,  13.4172,   0.7448],
        [ 49.5146, 113.7600,   0.7719,  13.3916,   0.7704],
        [102.1682,  18.2934,   2.3548,  13.4071,   0.7549],
        [ 81.0200,  10.2873,   9.3325,  13.4192,   0.7428],
        [ 65.0121, 105.9470,  39.1052,  13.4326,   0.7294],
        [ 37.5390,  44.1415,   1.3258,   1.2345,  12.9275],
        [117.9476,   4.5463,   9.5441,  13.4365,   0.7255],
        [148.2253,  17.0707,  27.8040,  

Helper functions for BO

In [13]:
bounds = torch.zeros(2, 8)
bounds[1] = 1

BATCH_SIZE = 4      # Number of candidates selected in each BO run/iteration
NUM_RESTARTS = 10   # Restarts during BO run
RAW_SAMPLES = 512   

ref_point = ref_point = torch.tensor([18, 0.1, 0.01], dtype=dtype)

In [16]:
from typing import Optional

def obj_callable(Z: torch.Tensor, X: Optional[torch.Tensor] = None):
    return IdentityMCMultiOutputObjective([0,1,2])

def constraint_callable(Z):
    class0 = Z[..., -2]
    class1 = Z[..., -1]
    return class1 - class0



In [14]:
def optimize_qnehvi_and_get_observation(model, train_x, train_obj, sampler):
    """Optimizes the qEHVI acquisition function, and returns a new candidate and observation."""
    with torch.no_grad():
        pred = model.posterior(normalize(train_x, x_bounds)).mean
    
    partitioning = FastNondominatedPartitioning(
        ref_point= ref_point,
        Y=pred,
    )

    acq_func = qNoisyExpectedHypervolumeImprovement(
        model=model,
        ref_point=ref_point,
        X_baseline=train_x,
        sampler=sampler,
        prune_baseline=True,
        objective=IdentityMCMultiOutputObjective(outcomes=[0, 1, 2]),
        constraints=[constraint_callable],
    )

    #optimize
    candidates, _ = optimize_acqf(
        acq_function=acq_func,
        bounds=bounds,
        q=BATCH_SIZE,
        num_restarts=NUM_RESTARTS,
        raw_samples=RAW_SAMPLES,
        options={"batch_limit":5, "maxiter": 200},
        sequential=True,
    )

    new_x = unnormalize(candidates.detach(), bounds=x_bounds)
    return new_x
    

In [17]:
sampler = SobolQMCNormalSampler(sample_shape=torch.Size([128]))

new_x_qnehvi = optimize_qnehvi_and_get_observation(model, x, y, sampler)

RuntimeError: shape mismatch: value tensor of shape [5] cannot be broadcast to indexing result of shape [12288, 3]

In [18]:
from botorch.test_functions.multi_objective import C2DTLZ2
tkwargs = {
    "dtype": torch.double,
    "device": torch.device("cuda:3" if torch.cuda.is_available() else "cpu"),
}
d = 4
M = 2
problem = C2DTLZ2(dim=d, num_objectives=M, negate=True).to(**tkwargs)
problem.ref_point.tolist()

[-1.100000023841858, -1.100000023841858]